# Multiple Chains

In [1]:
import os
from dotenv import load_dotenv

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
from operator import itemgetter

from langchain.schema import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt1 = ChatPromptTemplate.from_template("what is the city {person} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what country is the city {city} in? respond in {language}"
)

model = ChatOpenAI()

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"city": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

chain2.invoke({"person": "obama", "language": "german"})

'Barack Obama, der 44. Präsident der Vereinigten Staaten, wurde in Honolulu, Hawaii, geboren. Das Land ist die Vereinigten Staaten von Amerika.'

In [3]:
from langchain_core.runnables import RunnablePassthrough

prompt1 = ChatPromptTemplate.from_template(
    "generate a {attribute} color. Return the name of the color and nothing else:"
)
prompt2 = ChatPromptTemplate.from_template(
    "what is a fruit of color: {color}. Return the name of the fruit and nothing else:"
)
prompt3 = ChatPromptTemplate.from_template(
    "what is a country with a flag that has the color: {color}. Return the name of the country and nothing else:"
)
prompt4 = ChatPromptTemplate.from_template(
    "What is the color of {fruit} and the flag of {country}?"
)

model_parser = model | StrOutputParser()

color_generator = (
    {"attribute": RunnablePassthrough()} | prompt1 | {"color": model_parser}
)
color_to_fruit = prompt2 | model_parser
color_to_country = prompt3 | model_parser
question_generator = (
    color_generator | {"fruit": color_to_fruit, "country": color_to_country} | prompt4
)

In [4]:
question_generator.invoke("warm")

ChatPromptValue(messages=[HumanMessage(content='What is the color of Strawberry and the flag of Bhutan?')])

In [5]:
prompt = question_generator.invoke("warm")
model.invoke(prompt)

AIMessage(content='The color of "There is no known fruit named \'Coral\'" is not applicable as it is a statement. \n\nThe flag of Comoros consists of four horizontal stripes of yellow, white, red, and blue from top to bottom.')

## Branching and Merging

In [7]:
planner = (
    ChatPromptTemplate.from_template("Generate an argument about: {input}")
    | ChatOpenAI()
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

arguments_for = (
    ChatPromptTemplate.from_template(
        "List the pros or positive aspects of {base_response}"
    )
    | ChatOpenAI()
    | StrOutputParser()
)
arguments_against = (
    ChatPromptTemplate.from_template(
        "List the cons or negative aspects of {base_response}"
    )
    | ChatOpenAI()
    | StrOutputParser()
)

final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "Pros:\n{results_1}\n\nCons:\n{results_2}"),
            ("system", "Generate a final response given the critique"),
        ]
    )
    | ChatOpenAI()
    | StrOutputParser()
)

chain = (
    planner
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)

In [8]:
chain.invoke({"input": "scrum"})

"While Scrum has numerous benefits, it is important to acknowledge the potential cons or negative aspects of using this project management framework. Implementing Scrum can be complex and challenging, especially for teams that are new to agile methodologies. It requires a significant shift in mindset, roles, and processes, which can be difficult to grasp and adopt.\n\nAdditionally, Scrum's lack of structure can be a drawback for some teams. The framework does not provide a detailed step-by-step process, which means that teams may struggle with establishing clear guidelines and procedures. The iterative nature of Scrum can also lead to time and resource constraints, as sprints are time-boxed and require completing work within fixed timeframes. This can be challenging for projects with complex or uncertain requirements.\n\nFurthermore, Scrum's dependency on team collaboration can be both a benefit and a hindrance. While collaboration is essential, if team members are not effectively comm